In [581]:
class Mat:
    """Implementsção de uma matris
    """
    def __init__(self, rows, cols):
        #print(f"construtor do Mat invocado com {rows} e {cols}")
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]
    def numRows (self): return len(self.mat)
    def numCols (self): return len(self.mat[0])
    def __str__(self):
        "Devolve a matriz como uma string"
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)
    # def __repr__(self):
    #     "Utilizado no repl quando se pede para ver a matriz"
    #     return str(self)
    def __getitem__ (self, n):
        "Interface para a indexação []"
        return self.mat[n]

In [582]:
m = Mat(3,4)
m[0][1] = -5
m

In [583]:
class DotPlot(Mat):

    def __init__(self, seq1, seq2):
        super().__init__(len(seq1), len(seq2))
        self.s1 = seq1
        self.s2 = seq2
        self.dims = (len(seq1), len(seq2))

    def get_s1(self): 
        return self.s1
  
    def get_s2(self): 
        return self.s2

    # def __repr__(self):
    #     import io
    #     with io.StringIO() as s:
    #         print(' ', *list(self.s2), file = s)
    #         for x, L in zip(self.s1, self.mat):
    #             print(x, *L, file = s)
    #         return s.getvalue()


    def simple_dotplot(self):
        for L, x1 in enumerate(self.s1):
            for C, x2 in enumerate(self.s2):
                self.mat[L][C] = '*' if x1 == x2 else ' '
        import io
        with io.StringIO() as s:
            print(' ', *list(self.s2), file = s)
            for x, L in zip(self.s1, self.mat):
                print(x, *L, file = s)
            return print(s.getvalue())


    def extended_dotplot(self , window_size = 1, stringency = 1):     # Inspirado Prof Miguel Rocha
        start = int(window_size/2)  # isto para começar "centrado" e não haver erros de limites
        for L in range (start, len(self.s1) - start):
            for C in range(start, len(self.s2) - start):
                matches = 0
                l = C - start
                for k in range (L-start, L+start +1):
                    if self.s1[k] == self.s2[l]:
                        matches += 1
                    l += 1
                if matches >= stringency:
                    self.mat[L][C]= "*"
                else:
                    self.mat[L][C]= " "
        import io
        with io.StringIO() as s:
            print(' ', *list(self.s2), file = s)
            for x, L in zip(self.s1, self.mat):
                print(x, *L, file = s)
            return print(s.getvalue())

In [584]:
m = DotPlot("CGATATTAACGATT", "CGATATTAACGATT")
m.extended_dotplot(4,3)


  C G A T A T T A A C G A T T
C 0 0 0 0 0 0 0 0 0 0 0 0 0 0
G 0 0 0 0 0 0 0 0 0 0 0 0 0 0
A 0 0 *                 * 0 0
T 0 0   *                 0 0
A 0 0     *             * 0 0
T 0 0       *             0 0
T 0 0         *           0 0
A 0 0           *         0 0
A 0 0             *       0 0
C 0 0               *     0 0
G 0 0                 *   0 0
A 0 0 *   *             * 0 0
T 0 0 0 0 0 0 0 0 0 0 0 0 0 0
T 0 0 0 0 0 0 0 0 0 0 0 0 0 0



In [585]:
import io

def subst(x, y):

    import blosum as bl
    dic = dict(bl.BLOSUM(62))
    return int(dic[x+y])   # feito

                     
class NW:

  def __init__(self, s1, s2, g = -4):
    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)  # este + 1 por causa dos "cabeçalhos" com as seqs
    self.tr  = Mat(len(s1) + 1, len(s2) + 1)
    
    for L in range(len(s1)):                     # prencher a primeira linha
      self.mat[L + 1][0] = g * (L + 1)            # na matriz 
      self.tr[L + 1][0]  = 'C'                      # no Trace

    for C in range(len(s2)):                     # prencher a primeira coluna
      self.mat[0][C + 1] = g * (C + 1)             #na matriz
      self.tr[0][C + 1]  = 'E'                        # no trace

    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
        possiveis = [
            self.mat[L  ][C    ] + subst(x1, x2),   # Diagonal
            self.mat[L+1][C    ] + g,               # Esquerda
            self.mat[L  ][C + 1] + g,               # Cima
        ]
        dirs = "DEC"

        self.mat[L + 1][C + 1] = max(possiveis)   #colocar na matriz o valor max das 3 possibilidades
        self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]  # colocar D E C dependendo da posiçao onde foi obtido o max (self,mat[L +1][C + 1])

  def rebuild(self):
    # Começar no canto inferior direito
    L = len(self.s1)
    C = len(self.s2)
    S1 = ""
    S2 = ""
    
    dirs = {
        'D' : (-1, -1),
        'E' : ( 0, -1),
        'C' : (-1,  0)
    }

    while L > 0 or C > 0:
      # print(self.s1[L - 1], self.s2[C - 1])
      # print(L, C)
      # print(self.tr[L][C])
      DL, DC = dirs[self.tr[L][C]]

      if self.tr[L][C] == "D":
        S1 = self.s1[L - 1] + S1    # este self.s1/s2 representa as sequencias 
        S2 = self.s2[C - 1] + S2
      elif self.tr[L][C] == "E":
        S1 = '-' + S1
        S2 = self.s2[C - 1] + S2
      else:
        S1 = self.s1[L - 1] + S1
        S2 = '-' + S2        
      #print(S1, S2)
      L += DL
      C += DC

    # return S1, S2
    return print(S1, S2 , sep="\n")
 
  def max_score (self):
    return print("Max Score:", self.mat[len(self.s1)][len(self.s2)])

  def __repr__(self):
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, sep = '   ', file = S)           # Kinda COmplicado perceber mas isto permite fazer print das 2 matrizes Valores e Trace
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print(file = S)

      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return S.getvalue()

  def score_mat(self):
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, sep = '   ', file = S)           # Kinda COmplicado perceber mas isto permite fazer print das 2 matrizes Valores e Trace
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print(file = S)
      return print(S.getvalue())

  def trace_mat(self):
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return print(S.getvalue())



In [586]:
class SW:

  def __init__(self, s1, s2, g = -4):
    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)  # este + 1 por causa dos "cabeçalhos" com as seqs
    self.tr  = Mat(len(s1) + 1, len(s2) + 1)


    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
        possiveis = [
            self.mat[L  ][C    ] + subst(x1, x2),   # Diagonal
            self.mat[L+1][C    ] + g,               # Esquerda
            self.mat[L  ][C + 1] + g,               # Cima
            0]
        dirs = "DEC."

        self.mat[L + 1][C + 1] = max(possiveis)   #colocar na matriz o valor max das 3 possibilidades
        if self.mat[L + 1][C + 1] != 0:
          self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]  # colocar D E C dependendo da posiçao onde foi obtido o max (self,mat[L +1][C + 1])
  
  def max_score (self):
    max_score = 0
    linha = 0
    coluna = 0
    for L, x1 in enumerate(self.mat):
      for C, x2 in enumerate(self.mat):
        if self.mat[L][C] > max_score:
          max_score = self.mat[L][C]
          linha = L
          coluna = C
    return [max_score,linha,coluna]

  
  def rebuild(self):
    max_score = 0
    linha = 0
    coluna = 0
    for L, x1 in enumerate(self.mat):
      for C, x2 in enumerate(self.mat):
        if self.mat[L][C] > max_score:
          max_score = self.mat[L][C]
          linha = L
          coluna = C
    L = linha
    C = coluna
    S1 = ""
    S2 = ""
    
    dirs = {
        'D' : (-1, -1),
        'E' : ( 0, -1),
        'C' : (-1,  0),
        "." : (0, 0)
    }

    while L > 0 or C > 0:
      # print(self.s1[L - 1], self.s2[C - 1])
      # print(L, C)
      # print(self.tr[L][C])
      try:
        DL, DC = dirs[self.tr[L][C]]

        if self.tr[L][C] == "D":
          S1 = self.s1[L - 1] + S1    # este self.s1/s2 representa as sequencias 
          S2 = self.s2[C - 1] + S2
        elif self.tr[L][C] == "E":
          S1 = '-' + S1
          S2 = self.s2[C - 1] + S2
        elif self.tr[L][C] == "C":
          S1 = self.s1[L - 1] + S1
          S2 = '-' + S2  
        else:
          pass      
        L += DL
        C += DC
      except:
          return print(S1, S2 , sep="\n")

    # return S1, S2
    return print(S1, S2 , sep="\n")
 

        

    

  def __repr__(self):
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, sep = '   ', file = S)           # Kinda COmplicado perceber mas isto permite fazer print das 2 matrizes Valores e Trace
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print(file = S)

      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return S.getvalue()

  def score_mat(self):
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, sep = '   ', file = S)           # Kinda COmplicado perceber mas isto permite fazer print das 2 matrizes Valores e Trace
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print(file = S)
      return print(S.getvalue())

  def trace_mat(self):
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return print(S.getvalue())

In [587]:
alin = NW("PAWHEAE", "HEAGAWGHEE")
alin.score_mat()
alin.trace_mat()
alin.rebuild()
alin.max_score()




    -   H   E   A   G   A   W   G   H   E   E
-   0  -4  -8 -12 -16 -20 -24 -28 -32 -36 -40
P  -4  -2  -5  -9 -13 -17 -21 -25 -29 -33 -37
A  -8  -6  -3  -1  -5  -9 -13 -17 -21 -25 -29
W -12 -10  -7  -5  -3  -7   2  -2  -6 -10 -14
H -16  -4  -8  -9  -7  -5  -2   0   6   2  -2
E -20  -8   1  -3  -7  -8  -6  -4   2  11   7
A -24 -12  -3   5   1  -3  -7  -6  -2   7  10
E -28 -16  -7   1   3   0  -4  -8  -6   3  12


  - H E A G A W G H E E
- 0 E E E E E E E E E E
P C D D D E D E E E D D
A C D D D E D E E E E E
W C D C C D E D E E E E
H C D E D D D C D D E E
E C C D E E D C D C D D
A C C C D E D E D C C D
E C C D C D D E E D D D

--P-AW-HEAE
HEAGAWGHE-E
Max Score: 12
